In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
import xgboost as xgb

In [ ]:
train_data = pd.read_csv('train_DaEJRFg.csv')
test_data = pd.read_csv('test_TQDFDgg.csv')

In [ ]:
train_data.head()

In [ ]:
var = ['incident_tehsil', 'cause_of_emergency', 'roadway_feature', 'road_type']
for var in var:
    print (var)
    print train_data[var].value_counts()

In [ ]:
train_data.dtypes

In [ ]:
list(train_data.columns.values)

In [ ]:
for x in list(train_data.columns.values):
    print "Null values in " + str(x) + " are :" + str(sum(train_data[x].isnull()))

In [ ]:
for col in list(train_data.columns.values):
    print "Unique values in " + str(col) + " are :" + str(len(train_data[col].unique()))

In [ ]:
train_data.corr()

In [ ]:
data = train_data[train_data['criticality']==1]
print len(data)
print len((data['incident_location'].unique()))

In [ ]:
train_data.describe(include='all')

In [ ]:
train_data['hour'] = train_data['incident_time'].str[:2].astype(int)
train_data['minute'] = train_data['incident_time'].str[3:5].astype(int)
train_data['second'] = train_data['incident_time'].str[6:].astype(int)

test_data['hour'] = test_data['incident_time'].str[:2].astype(int)
test_data['minute'] = test_data['incident_time'].str[3:5].astype(int)
test_data['second'] = test_data['incident_time'].str[6:].astype(int)

In [ ]:
train_data['date'] = train_data['incident_date'].str[:2].astype(int)
train_data['month'] = train_data['incident_date'].str[3:5].astype(int)
train_data['year'] = train_data['incident_date'].str[6:].astype(int)

test_data['date'] = test_data['incident_date'].str[:2].astype(int)
test_data['month'] = test_data['incident_date'].str[3:5].astype(int)
test_data['year'] = test_data['incident_date'].str[6:].astype(int)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("ISBT raipur dehradun") 
print(location.address)
print((location.latitude, location.longitude))

In [ ]:
train_data['surrounding_area'].unique()

In [ ]:
train_data.incident_tehsil.replace(['Raipur', 'Sahaspur', 'Dehradun', 'Doiwala', 'Vikasnagar',
       'Chakrata', 'Kalsi'], [1, 2, 3, 4, 5, 6, 7], inplace=True)
train_data.cause_of_emergency.replace(['Multiple Vehicular Incident', '2 Wheeler accidents',
       'Non motorised vehicle accidents(Bullock cart,Bicycle,Rickshaw,Push cart)',
       'MVC - Pedestrian (Run Over/Hit & Run)', 'Vehicle role over/Skid',
       'Auto (3 Wheeler accidents)', 'MVC-Thrown off from Vehicle',
       'MVC-Fixed object (Divider)', 'MCI',
       'Vehicle -Fall From Height etc.', 'Railroad Train/Other Object',
       'MVC-Non traffic inside a parked vehicle/Skidding', 'MVC-MV',
       'Extrication>20mts'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace=True)
train_data.road_type.replace(['HIW', 'LOC'], [1, 2], inplace=True)
train_data.roadway_feature.replace(['CUR', 'INT', 'SLP', 'UNK', 'STR'], [1, 2, 3, 4, 5], inplace=True)
train_data.surrounding_area.replace(['FOR', 'AGL', 'HOSP', 'HIL', 'MAR', 'COL'], [1, 2, 3, 4, 5, 6], inplace=True)

In [ ]:
test_data.incident_tehsil.replace(['Raipur', 'Sahaspur', 'Dehradun', 'Doiwala', 'Vikasnagar',
       'Chakrata', 'Kalsi'], [1, 2, 3, 4, 5, 6, 7], inplace=True)
test_data.cause_of_emergency.replace(['Multiple Vehicular Incident', '2 Wheeler accidents',
       'Non motorised vehicle accidents(Bullock cart,Bicycle,Rickshaw,Push cart)',
       'MVC - Pedestrian (Run Over/Hit & Run)', 'Vehicle role over/Skid',
       'Auto (3 Wheeler accidents)', 'MVC-Thrown off from Vehicle',
       'MVC-Fixed object (Divider)', 'MCI',
       'Vehicle -Fall From Height etc.', 'Railroad Train/Other Object',
       'MVC-Non traffic inside a parked vehicle/Skidding', 'MVC-MV',
       'Extrication>20mts'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace=True)
test_data.road_type.replace(['HIW', 'LOC'], [1, 2], inplace=True)
test_data.roadway_feature.replace(['CUR', 'INT', 'SLP', 'UNK', 'STR'], [1, 2, 3, 4, 5], inplace=True)
test_data.surrounding_area.replace(['FOR', 'AGL', 'HOSP', 'HIL', 'MAR', 'COL'], [1, 2, 3, 4, 5, 6], inplace=True)

In [ ]:
train_data['Date']=pd.to_datetime((train_data.year*10000+train_data.month*100+train_data.date).apply(str),format='%Y%m%d')

test_data['Date']=pd.to_datetime((test_data.year*10000+test_data.month*100+test_data.date).apply(str),format='%Y%m%d')

In [ ]:
train_data['weekday'] = train_data['Date'].dt.dayofweek
test_data['weekday'] = test_data['Date'].dt.dayofweek

In [ ]:
train_data['season'] = train_data['month']
test_data['season'] = test_data['month']

In [ ]:
train_data['season']=train_data['season'].replace([11,12,1,2], 1)
train_data['season']=train_data['season'].replace([3,4,5,6], 2)
train_data['season']=train_data['season'].replace([7,8,9,10], 3)

test_data['season']=test_data['season'].replace([11,12,1,2], 1)
test_data['season']=test_data['season'].replace([3,4,5,6], 2)
test_data['season']=test_data['season'].replace([7,8,9,10], 3)

In [ ]:
train_data['hour_bin'] = train_data['hour']
test_data['hour_bin'] = test_data['hour']

In [ ]:
train_data['hour_bin']=train_data['hour_bin'].replace([0,1,2,3,4,5], 1)
train_data['hour_bin']=train_data['hour_bin'].replace([6,7,8,9,10,11], 2)
train_data['hour_bin']=train_data['hour_bin'].replace([12,13,14,15,16,17], 3)
train_data['hour_bin']=train_data['hour_bin'].replace([18,19,20,11,22,23], 4)

test_data['hour_bin']=test_data['hour_bin'].replace([0,1,2,3,4,5], 1)
test_data['hour_bin']=test_data['hour_bin'].replace([6,7,8,9,10,11], 2)
test_data['hour_bin']=test_data['hour_bin'].replace([12,13,14,15,16,17], 3)
test_data['hour_bin']=test_data['hour_bin'].replace([18,19,20,11,22,23], 4)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
X = train_data.drop(['victim_id','incident_time','incident_date','incident_location','criticality','Date', 'second', 'minute', 'hour', 'month'] ,axis=1)
Y = train_data['criticality']

In [ ]:
X_test = test_data.drop(['victim_id','incident_time','incident_date','incident_location','Date', 'second', 'minute','hour', 'month'], axis=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.20)

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=1000,
                   max_depth=1,
                   min_child_weight=11,
                   learning_rate=0.1,
                   subsample=0.75,
                   colsample_bytree=0.9,
                   gamma=0.0,
                   objective='binary:logistic',
                   nthread=4,
                   scale_pos_weight=1,
                   seed=7
                  )

xgb_model.fit(x_train, y_train)

pred1 = xgb_model.predict(x_train)
pred2 = xgb_model.predict(x_test)

In [ ]:
predict_prob = xgb_model.predict_proba(X_test)

In [ ]:
z = predict_prob[:, 1]

In [ ]:
submission = pd.DataFrame(test_data['victim_id'])
submission['criticality'] = z

In [ ]:
submission.head()

In [ ]:
submission.to_csv('aplha_omega_submission.csv', index=False)

In [ ]:
xgb.plot_importance(xgb_model, height=4, xlabel='Importance', ylabel='Features')